In [1]:
# Initialization

%matplotlib inline
import matplotlib.pyplot as plt, numpy as np, pandas as pd
from IPython.display import display
from lesson1 import figure

1) Interpolation

Misc

# 1) Interpolation
When only a limited set of discrete values $(x_i, y_i)$ are available, the original continuous signal $y(x)$ may be approximated at intermediate values $x$ by interpolation. 

**Polynomial functions** for short signals.<br>
**Piecewise functions** for long signals.<br>
**Trigonometric function** for quasi-periodic signals.


- a) Polynomial; global interpolation: uses all $n$ data points.
- b) Spline/piecewise; local interpolation: uses a subset of nearby points.

When to use what? <br>
High-degree polynomials result in unfaithful interpolation due to Runge's phenomenon. The use of a **piecewise polynomial** also known as a **spline**, allows the degree of employed polynomials to be reduced. Oscillations are much reduced.

### a) Polynomial 

$$
y(x) = a_p \cdot x^p + a_{p-1} \cdot x^{p-1} + \ldots + a_2 \cdot x^2 + a_1 \cdot x + a_0
$$


- If interpolated data containes negative values that are unrealistic, model the logarithm, $log(y)$, as a polynomial function of $x$.


In [7]:
xi = np.arange(3, 24, 2)
yi = np.array([
    92, 7, 16, 885, 849,
    1399, 1590, 1261, 5690, 670, 3271
])

#### i) lagrange

In [6]:
from scipy.interpolate import lagrange
"""
f.c will return the polynomial coefficients [a_p, ..., a_1, a_0]
"""

f = lagrange(xi, yi) 
f(12) 

1000.3715858738869

#### ii) vander_interpolate

In [12]:
def vander_interpolate(xi, yi, x):
    """y = vander_interpolate(xi, yi, x).
    Vandermonde interpolation method that fits a
    polynomial of degree n-1 through n data points
    {xi,yi}, evaluated at arguments x.
    xi     = {x1,x2,...,xn}
    yi     = {y1,y2,...,xn}
    x      = arguments x
    """
    if xi.size != yi.size:
        raise ValueError('xi and yi must have the same length')
    
    xi = xi.astype('float64')
    yi = yi.astype('float64')
    
    X = np.vander(xi)
    a = np.linalg.solve(X, yi)
    
    y = np.polyval(a, x)
    
    return y

In [14]:
def rational_interpolate(xi, yi, x, *, p=None, q=None):
    """
    Benefit: typically smoother and less oscillatory than polynomials 
    and are not required to go to infinity for extreme values of x.
    Downside: can introduce undesirable asymptotes in the fit.
    
    y = rational_interpolate(xi, yi, x, *, p=None, q=None).
    Rational interpolation method that fits a rational
    function of polynomial degrees p and q through n data
    points {xi,yi}, evaluated at arguments x. If neither p
    nor q are provided, a diagonal rational function is used.
    xi     = {x1,x2,...,xn}
    yi     = {y1,y2,...,xn}
    x      = arguments x
    p      = polynomial degree numerator
    q      = polynomial degree denominator
    """
    if p is None:
        if q is None:
            q = yi.size // 2
        p = yi.size - q - 1
    elif q is None:
        q = yi.size - p - 1
    if xi.size != yi.size:
        raise ValueError('xi and yi must have the same length')
    if yi.size != p + q + 1:
        raise ValueError('number of data points must equal p+q+1')
        
    xi = xi.astype('float64')
    yi = yi.astype('float64')
    
    n = p + q + 1
    X = np.zeros((n, n))

    # index 0-based, all columns set to 1
    middle = p
    
    # create p-degree values
    p_vals = []
    for val in xi:
        temp = []
        for degree in range(p, 0, -1):
            p_val = val**degree
            temp.append(p_val)

        p_vals.append(temp)

    p_vals = np.array(p_vals)
    
    # create q-degree values
    q_vals = []
    for _x, _y in zip(xi, yi):
        temp = []
        for degree in range(q, 0, -1):
            q_val = -1 * (_x**degree * _y)
            temp.append(q_val)

        q_vals.append(temp)

    q_vals = np.array(q_vals)

    X[:, middle] = 1
    X[:, :p] = p_vals
    X[:, middle+1:] = q_vals
    
    coef = np.dot(np.linalg.inv(X), yi)
        
    a = coef[:p]
    mid = coef[p]
    b = coef[p+1:]

    p_vander = np.vander(x, p+1)[:, :-1]
    q_vander = np.vander(x, q+1)[:, :-1]

    y = ((a * p_vander).sum(axis=1) + mid) / ((b * q_vander).sum(axis=1) + 1)    
    return y

## b) Spline/piecewise
The coefficients of the piecewise polynomials $a_{ij}$ can be fixed by imposing a series of *continuity constraints*.

1. **Left-continuity:** At $x_i$, the polynomial $p_i(x)$ should evaluate to $y_i$.<br />
   I.e., each segment should connect to the data point $(x_i, y_i)$ on the left.

2. **Right-continuity:** At $x_{i+1}$, the polynomial $p_i(x)$ should evaluate to $y_{i+1}$.<br />
   I.e., each segment should connect to the data point $(x_{i+1}, y_{i+1})$ on the right.

3. **Straightness:** At $x_i$, the 1<sup>st</sup> derivative of the left polynomial $\frac{\text{d}}{\text{d}x} p_{i-1}(x)$ should match the 1<sup>st</sup> derivative of the right polynomial $\frac{\text{d}}{\text{d}x} p_i(x)$.<br />
   I.e., the segments should connect in a straight fashion.

4. **Smoothness:** At $x_i$, the 2<sup>nd</sup> derivative of the left polynomial $\frac{\text{d}^2}{\text{d}x^2} p_{i-1}(x)$ should match the 2<sup>nd</sup> derivative of the right polynomial $\frac{\text{d}^2}{\text{d}x^2} p_i(x)$.<br />
   I.e., the segments should connect in a smooth fashion.
   
   
* A zero spline is fully defined by the first constraint alone.<br />
  It generally does not satisfy the other constraints: the interpolant is discontinuous.

* A linear spline is fully defined by the first and second constraints.<br />
  It generally does not satisfy the straightness and smoothness constraints: the interpolant is continuous but its slope is not.

* A quadratic spline is fully defined by the first to third constraints.<br />
  It generally does not satisfy the smoothness constraint: the interpolant's slope is continuous but its curvature is not.

* A cubic spline is fully defined by the first to fourth constraints.<br />
  The interpolant's slope and curvature are continuous.


#### i) interp1d

In [11]:
from scipy.interpolate import interp1d
"""
Kind: [linear, nearest, zero, linear, quadratic, cubic, ...]
fill value: used for extrapolation
"""
f = interp1d(xi, yi, kind='cubic', fill_value='extrapolate')
f(12)

array(1029.30950272)

**ii) Akima1DInterpolator**

`Akima1DInterpolator()` fits a non-smoothing [Akima](https://en.wikipedia.org/wiki/Akima_spline) cubic spline that is able to deal more robustly with functions with rapidly varying curvature, at the cost of no longer satisfying the smoothness constraint.
  
<small>**Note:** this spline function does not extrapolate.</small>

In [ ]:
from scipy.interpolate import Akima1DInterpolator

spline = Akima1DInterpolator(xi, yi)
x = np.linspace(-5.5, 5.5, 111)
y = spline(x)

**iii) CubicSpline**

fits a [cubic Hermite spline](https://en.wikipedia.org/wiki/Cubic_Hermite_spline) with selectable types of boundary conditions.

  * `bc_type='not-a-knot'`: the first and second segment are the same polynomial.
  
  * `bc_type='periodic'`: the function value and its derivatives at the last knot match those at the first knot.
  
  * `bc_type='clamped'`: the 1<sup>st</sup> derivatives at the first and last knot equal zero.
  
  * `bc_type='natural'`: the 2<sup>nd</sup> derivatives at the first and last knot equal zero.

In [17]:
from scipy.interpolate import CubicSpline

spline = CubicSpline(xi, yi, bc_type=bc_type)
y = spline(x)

**iv) PchipInterpolator**

`PchipInterpolator()` fits a *Piecewise Cubic Hermite Interpolating Polynomial* spline that does not introduce additional extrema (i.e., minima or maxima) except those that coincide with knots.

In [18]:
from scipy.interpolate import PchipInterpolator

spline = PchipInterpolator(xi, yi)
x = np.linspace(-5.5, 5.5, 111)
y = spline(x)

**v) CubicHermiteSpline**

`CubicHermiteSpline()` fits a cubic spline for which derivatives at knots are specified in addition to function values.

In [ ]:
from scipy.interpolate import CubicHermiteSpline

spline = CubicHermiteSpline(xi, yi, dyi)
x = np.linspace(-5.5, 5.5, 111)
y = spline(x)

**vi) Custom**

In [ ]:
def nearest_interpolate(xi, yi, x):
    """
    Alternative:  interp1d(x, y, kind='nearest')
    
    y = nearest_interpolate(xi, yi, x).
    Nearest-neighbor spline interpolation method that
    fits a piecewise polynomial of degree zero through
    data points {xi,yi}, evaluated at arguments x.
    xi     = {x1,x2,...,xn}
    yi     = {y1,y2,...,xn}
    x      = arguments x
    """
    if xi.size != yi.size:
        raise ValueError('xi and yi must have the same length')
    
    xi = xi.astype('float64')
    yi = yi.astype('float64')
    
    order = np.argsort(xi)
    xi, yi = xi[order], yi[order]
    xknot = (xi[:-1] + xi[1:]) / 2.0
    index = np.searchsorted(xknot, x)
    y = yi[index]
    return y

def cubic_interpolate(xi, yi, x):
    """
    Uniformly weighted cubic spline
    
    y = cubic_interpolate(xi, yi, x).
    Cubic spline interpolation method that fits a
    piecewise polynomial of degree three through
    data points {xi,yi}, evaluated at arguments x.
    xi     = {x1,x2,...,xn}
    yi     = {y1,y2,...,xn}
    x      = arguments x
    """
    if xi.size != yi.size:
        raise ValueError('xi and yi must have the same length')
        
    xi = xi.astype('float64')
    yi = yi.astype('float64')
        
    order = np.argsort(xi)
    xi, yi = xi[order], yi[order]

    xknots = xi
    idx = np.searchsorted(xknots, x)

    # left parabola
    if idx-2 == -1:
        left_par_xi = xi[idx-1:idx+1]
        left_par_yi = yi[idx-1:idx+1]
    elif idx-2 == -2:
        left_par_xi = xi[idx:idx+1]
        left_par_yi = yi[idx:idx+1]
    else:
        left_par_xi = xi[idx-2:idx+1]
        left_par_yi = yi[idx-2:idx+1]

    left_par_f = lagrange(left_par_xi, left_par_yi)
    left_par_x = left_par_f(x) 

    # right parabola
    right_par_xi = xi[idx-1:idx+2]
    right_par_yi = yi[idx-1:idx+2]

    right_par_f = lagrange(right_par_xi, right_par_yi)
    right_par_x = right_par_f(x) 

    # calculate applicable weight for left and right
    left_weight = xi[idx] - x 
    right_weight = x - xi[idx-1] 

    # uniform weight
    y = 0.5 * left_par_x + 0.5 * right_par_x
    
    return y

# Misc

In [15]:
def P(n):
    """Square pyramidal numbers"""
    t = 0
    for k in range(n+1):
        t += k**2
        
    return t

def Fibonacci(n):
    """Fibonacci sequence"""
    if n < 0:
        print("Incorrect input")
    elif n == 0:
        return 0
    elif n == 1 or n == 2:
        return 1
 
    else:
        return Fibonacci(n-1) + Fibonacci(n-2)
    
def factorial(n):
    total = 1
    for k in range(1, n+1):
        total *= k
        
    return total

def C(n):
    """Catalan numbers"""
    if n == 0 or n == 1:
        return 1
    
    total = 1
    for k in range(2, n+1):
        total *= (n+k)/k
        
    return total
